En este notebook se mostrará el código referente al entrenamiento del modelo, así como del análisis de resultados y ejecución del programa con el modelo generado.

En primer lugar, se muestra el código utilizado para el entrenamiento de nuestro modelo:

In [1]:
# Importaciones
!pip3 install peft
!pip3 install scikit-learn
!pip3 install datasets

import os
import pandas as pd
import numpy as np
import peft
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from peft import LoraConfig
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset, Dataset
from huggingface_hub import snapshot_download


[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB 2.0 MB/s eta 0:00:05
    --------------------------------------- 0.1/8.9 MB 1.4 MB/s eta 0:00:07
   - -------------------------------------- 0.3/8.9 MB 2.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.9 MB 3.0 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/8.9 MB 3.5 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.4/8.9 MB 4.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.9 MB 4.7 MB/s eta 0:00:02
   -------- ------------------------------- 1.9/8.9 MB 4.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/8.9 MB 4.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.3/8.9 MB 4.5 MB/s eta 0:00:02
   ----------- ---------------------------- 2.5/8.9 MB 4.7 MB/s eta 0:00:02
   ----------- ----


[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Realizadas las importaciones, se procede a la descarga del modelo pre-entrenado especializado en clasificación de secuencias

In [ ]:
nombre_modelo = 'PlanTL-GOB-ES/roberta-large-bne-massive'
snapshot_download(repo_id=nombre_modelo, cache_dir='./huggingface_mirror')

Se procede a la apertura del dataframe con los datos de entrenamiento y se generan las etiquetas, que serán las clases del modelo supervisado, y sus números asociados, pues los Transformers entienden números, no textos:

In [3]:
nombre_fichero = os.getcwd() + '\\ficheros_excel\\MODELO_ENTRENAMIENTO.xlsx'
df_modelo_entrenamiento = pd.read_excel(io=nombre_fichero, sheet_name='MODELO_ENTRENAMIENTO', index_col=None)

# Renombramos las columnas, ya que de lo contrario puede dar errores durante el entrenamiento
df_modelo_entrenamiento.columns = ['texto', 'etiqueta']
etiquetas = df_modelo_entrenamiento['etiqueta'].tolist()
numero_de_etiquetas = len(etiquetas)

# Las etiquetas deben ser números dentro del transformer y se han de crear mapas de los ids/identificadores esperados con id2label y label2id
etiquetas_a_id = {}
ids_a_etiquetas = {}
i = 0
while i < len(etiquetas):
    ids_a_etiquetas[i] = etiquetas[i]
    etiquetas_a_id[etiquetas[i]] = i
    i = i + 1

df_modelo_entrenamiento['etiqueta'] = ids_a_etiquetas.keys()


Para adaptar el modelo pre-entrenado al que se entrenará, resulta necesaria la modificación de la capa de clasificación, lo cual se realiza a continuación:

In [4]:
# Creamos la configuración para modificar la capa de clasificación de nuestro modelo preentrenado al de nuestro modelo
nueva_configuracion_modelo = AutoConfig.from_pretrained(nombre_modelo, num_labels=numero_de_etiquetas, id2label=ids_a_etiquetas, label2id=etiquetas_a_id, cache_dir='./huggingface_mirror')


Se obtienen a continuación el tokenizador y modelo ya pre-entrenados:

In [5]:
# Obtenemos el modelo y tokenizador del modelo ya preentrenado
modelo_roberta = AutoModelForSequenceClassification.from_pretrained('PlanTL-GOB-ES/roberta-large-bne-massive', cache_dir='./huggingface_mirror', local_files_only=True)#, num_labels=num_labels, id2label=ids_a_etiquetas, label2id=etiquetas_a_id)

if modelo_roberta.config.num_labels != nueva_configuracion_modelo.num_labels or modelo_roberta.config.id2label != nueva_configuracion_modelo_config.id2label:
    modelo_roberta.classifier.out_proj.out_features=nueva_configuracion_modelo.num_labels
    
modelo_roberta.config = nueva_configuracion_modelo

tokenizador_roberta = AutoTokenizer.from_pretrained(nombre_modelo, cache_dir='./huggingface_mirror', local_files_only=True, from_pt=True)

Se procede a continuación a la carga del dataset, separando los datos para entrenamiento, test y validación. Una vez hecho, se hará la tokenización de los atributos, es decir, se transformarán las palabras en números o, dicho de otro modo, datos entendibles por la arquitectura utilizada para el entrenamiento:

In [10]:
def tokenizador_texto(textos, tokenizador):
    texto = textos['texto']
    textos_tokenizados = tokenizador(texto, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    return textos_tokenizados

# Cargamos el dataset y separamos en distintas variables los valores de entrenamiento y de test
dataset = Dataset.from_pandas(df_modelo_entrenamiento, split='train')
dataset = dataset.train_test_split(test_size=0.3, shuffle=True, seed=42)

if tokenizador_roberta.pad_token is None:
    tokenizador_roberta.add_special_tokens({'pad_token': '[PAD]'})
    modelo_roberta.resize_token_embeddings(len(tokenizador_roberta))

dataset_tokenizado = dataset.map(tokenizador_texto, batched=True, fn_kwargs={'tokenizador': tokenizador_roberta})

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 2656.95 examples/s]


Con las instancias ya separadas por tipo, se procede a realizar la aumentación de los datos de entrenamiento:

Generaremos a continuación un modelo PEFT mediante LoRA. Esto nos permite la congelación de los pesos del modelo pre-entrenado que, además de reducir los tiempos de entrenamiento, facilita unos mejores resultados cuando los datos de entrenamiento son escasos como aquí sucede

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizador_roberta, padding=True)
#accuracy = evaluate.load("accuracy")

peft_config = LoraConfig(task_type='TOKEN_CLS', r=4, lora_alpha=32, lora_dropout=0.01, target_modules=['query'])
peft_model = peft.get_peft_model(model=modelo_roberta, peft_config=peft_config)

peft_model.print_trainable_parameters()

Configuramos los hiperparámetros y otras variables de entrenamiento:

In [ ]:
def obtener_metricas(p):
    predicciones, etiquetas = p
    predicciones = np.argmax(predicciones, axis=1)
    accuracy = accuracy_score(etiquetas, predicciones)
    precision = precision_score(etiquetas, predicciones, average='weighted')
    recall = recall_score(etiquetas, predicciones, average='weighted')
    f1 = f1_score(etiquetas, predicciones, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir=nombre_modelo + '-custom-lora',
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    optim='adamw_torch',
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset_tokenizado['train'],
    eval_dataset=dataset_tokenizado['test'],
    tokenizer=tokenizador_roberta,
    data_collator=data_collator,
    compute_metrics=obtener_metricas
)

Se procede ahora al entrenamiento y evaluación de resultados:

In [ ]:
trainer.train()
peft_model.to('cpu')
peft_model.eval()